In [1]:
import pickle
from rdflib import Graph
import torch
from torch_geometric.data import Data
import gzip
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary
from torch_geometric.nn import GCNConv, BatchNorm,  TopKPooling, global_add_pool, global_max_pool
from tqdm import tqdm


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [3]:
#Load the rdf graph and form it into an edge_index
g = Graph()
g.parse("data/rdf_string.ttl", format="turtle")
seen={}
ei=[[],[]]
ii=0
def get_name(raw):
    index=raw.rfind('_')
    return raw[index+1:-1]
for i,j,k in g:
    sbj=get_name(i.n3())
    obj=get_name(k.n3())
    if sbj not in seen:
        seen[sbj]=ii
        ii+=1
    if obj not in seen:
        seen[obj]=ii
        ii+=1
    ei[0].append((seen[sbj]))
    ei[1].append((seen[obj]))

In [5]:
#labels
import gzip
f=gzip.open('/encrypted/e3001/robert/morbid.txt.gz','rt')
lines=f.readlines()
f.close()
phenos={}
labels={}
i=0
for l in lines:
    l=l.rstrip().replace('"','')
    p=l.split('\t')
    if len(p[-1])<2:
        labels[p[0]]=p[1:-1]
    else:
        labels[p[0]]=p[1:]
    for x in labels[p[0]]:
        phenos[x]=1

In [6]:
def get_codes(dis):
    result={}
    for p in phenos:
        if dis in p:
            result[p]=True
    return [x for x in result]
def intersection(lst1, lst2): 
    temp = set(lst1) 
    lst3 = [value for value in lst2 if value in temp]
    return lst3 
def get_by_disease(dis,upb):
    maxi=0
    positive=0
    negative=0
    pos={}
    neg={}
    for p in tqdm(labels):
        if positive>upb:
            break
        if len(intersection(dis,labels[p]))>0:
            positive+=1
            pos[p+'_'+p]=True
        elif positive>negative:
            negative+=1
            neg[p+'_'+p]=True
    return pos,neg
#get all codes of breast cancer
codes=get_codes('C50')
#get dataset of disease and upperbound on number of positive cases
pos,neg=get_by_disease(codes,3000000)
print(len(pos))


100%|██████████| 502616/502616 [00:00<00:00, 759502.15it/s]

12410


In [7]:
#get data but only of the patients in pos and neg
#dictionary to convert from NCBI gene to ENS
f=open('toENS.pkl','rb')
dic=pickle.load(f)
f.close()
#Load individuals' data
f=open('files.txt')
lines=f.readlines()
f.close()
notfound={}
feat_vecs={}
dataset=[]
for l in lines:
    print(l)
    f=open(l.rstrip(),'rb')
    pe=pickle.load(f)
    for p in tqdm(pe):
        if p not in neg and p not in pos:
            continue
        if p not in feat_vecs:
            feat_vecs[p]=[[0] for i in range(len(seen))]
        val={}
        val[p]=[1 for i in range(len(seen))]
        for g in pe[p]:
            if g[-1][0] not in dic:
                notfound[g[-1][0]]=True
            else:
                prots=dic[g[-1][0]]
                for prot in prots:
                    if prot=='-':
                        continue
                    index=prot.find('.')
                    if index>-1:
                        prot=prot[:index]
                    if prot in seen and float(g[8])<val[p][seen[prot]]:
                        feat_vecs[p][seen[prot]][0]=float(g[9])#float(g[8])
                        val[p][seen[prot]]=float(g[8])
                        #feat_vecs[p][seen[prot]][1]=float(g[9])
                        #feat_vecs[p][seen[prot]][2]=float(g[10])

/encrypted/e3001/awesome_sumyyah/random/chr10_final.pkl



100%|██████████| 488224/488224 [03:03<00:00, 2654.21it/s]


/encrypted/e3001/awesome_sumyyah/random/chr11_final.pkl



100%|██████████| 139991/139991 [00:03<00:00, 39246.60it/s]


/encrypted/e3001/awesome_sumyyah/random/chr12_final.pkl



...

100%|██████████| 486134/486134 [00:12<00:00, 39166.91it/s]


/encrypted/e3001/awesome_sumyyah/random/chr9_final.pkl



100%|██████████| 352912/352912 [00:08<00:00, 39702.65it/s]
0it [00:00, ?it/s]


/encrypted/e3001/awesome_sumyyah/random/chrY_final.pkl

/encrypted/e3001/awesome_sumyyah/random/chrX_final.pkl



100%|██████████| 484705/484705 [00:13<00:00, 35929.26it/s]


In [8]:
dataset=[]
test_set=[]
test_set_labels=[]
#create dataset of batches
tpos=sum([1 for t in feat_vecs if t in pos])
tneg=len(feat_vecs)-tpos
train_pos=int(tpos*.90)
train_neg=int(tneg*.90)
i=0
batch=[]
batch_l=[]
currp=0
currn=0
test_p=False
test_n=False
for t in feat_vecs:
    label=0.0
    if t in pos:
        label=1.0
    if currp==train_pos and label>0:
        test_p=True
    if currn==train_neg and label<1:
        test_n=True
    if i%32==0 and i!=0:
        if len(batch)>1:
            dataset.append([batch,batch_l])
            batch=[]
            batch_l=[]
    if test_p and label>0:
        test_set.append(feat_vecs[t])
        test_set_labels.append(label)
    elif test_n and label<1:
        test_set.append(feat_vecs[t])
        test_set_labels.append(label)
    else:
        batch.append(feat_vecs[t])
        batch_l.append(label)
        if label>0:
            currp+=1
        else:
            currn+=1
    i+=1
dataset.append([batch,batch_l])
print(len(batch),len(dataset)*32,len(test_set))

1 21408 2375


In [ ]:
from torch_geometric.data import Data
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = GCNConv(1,5)
        self.BN1 = BatchNorm(5)
        self.pool1=  TopKPooling(5,0.8)
        self.conv2 = GCNConv(5,8)
        self.BN2 = BatchNorm(8)
        self.pool2=  TopKPooling(8, 0.0001)
        self.fc1 = nn.Linear(16,1)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x=self.conv1(x, edge_index)
        x = F.relu((x),inplace=True)
        x = F.relu((self.conv2(x, edge_index)),inplace=True)
        x, edge_index, _, _, _, _ = self.pool2(x, edge_index)
        x = x.view(1,-1)
        x = self.fc1(x)
        x=torch.sigmoid(x)
        return x

In [ ]:
def nonzero(x):
    count=0
    for i in x:
        if i>0:
            count+=1
    return count
#torch.cuda.empty_cache()
import matplotlib.pyplot as plt
from tqdm import tqdm
loss = nn.BCELoss()
device='cuda'
#loss = nn.CrossEntropyLoss()
loss = loss.to(device)

epochs=1
losat=[]
trial = MyNet()
tria=trial.to(device)
optimizer = optim.SGD(trial.parameters(), lr = 0.001)
trial.train()
trues=[]
for e in range(epochs):
    #optimizer.zero_grad()
    local=[]
    predictions=[]
    print('epoch:',e)
    i=0
    for b in tqdm(dataset):
        if len(b[0])<1:
            break
        loss_train=0
        for i in range(len(b[0])):
            #print(i,sum([x[0] for x in b[0][i]]),(b[1][i]))
            data=Data(x=torch.tensor(b[0][i],dtype=torch.float),edge_index=torch.tensor(ei, dtype=torch.long),y=torch.tensor([[b[1][i]]]))
            data=data.to(device)
            preds = trial(data)
            #print(preds,nonzero(data.x),data.y)
            predictions.append(preds.detach())
            loss_train += loss(preds,torch.tensor(data.y,dtype=torch.float).to(device))
            local.append(loss_train.detach())
        #print(preds)
        #print(torch.cuda.memory_allocated())
        optimizer.zero_grad()
        loss/=len(b[0])        
        loss_train.backward()
        optimizer.step()
        #print(torch.cuda.memory_allocated())
        #print(trial(data))
    preds=0
    data=0
    plt.plot(local, label = 'training loss',alpha=0.4)
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()